<a href="https://colab.research.google.com/github/BALaka-18/Decision-Trees-Series/blob/master/Decision_Trees1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

## TASK 1 : 
You have to work on the play tennis(https://www.kaggle.com/fredericobreno/play-tennis) dataset and manually find out on which column the first split will take place.

**Hint:** Find Information Gains for all the columns

In [46]:
data = pd.read_csv('/content/drive/My Drive/play_tennis.csv')
data.head()

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   day       14 non-null     object
 1   outlook   14 non-null     object
 2   temp      14 non-null     object
 3   humidity  14 non-null     object
 4   wind      14 non-null     object
 5   play      14 non-null     object
dtypes: object(6)
memory usage: 800.0+ bytes


#### Function for Calculating entropy

In [0]:
def entropy_cont(label_col,data=iris):
  prob = data[label_col].value_counts()/data[label_col].value_counts().sum()
  entr = sum([-p*np.log(p) for p in prob])
  return entr

#### The play_tennis dataset (decision trees on purely categorical data)

#### Function to find best split based on information gain value

In [0]:
# Function to find out most valuable column by calculating highest information gain iteratively.
def best_split(data):
    info_gain = []
    yes_count = data[data.columns[len(data.columns)-1]].value_counts()['Yes']   # No. of 'Yes'
    no_count = data[data.columns[len(data.columns)-1]].value_counts()['No']     # No. of 'No'
    parent = entropy_cont('play',data=data)               # Calculating the parent entropy
    for i in data.columns[1:-1]:
        # Grouping on column
        grouped = data.groupby(i).count()[data.columns[len(data.columns)-1]]
        name_list = grouped.index.tolist()  # Get the unique values in a column
        no_list = grouped.tolist()          # Get the value count.
        child = 0
        for j in range(len(name_list)):  # Calculate entropy for each child node.
            data_temp = data[data[i]==name_list[j]]
            try:
                yes_count1 = data_temp[data.columns[len(data.columns)-1]].value_counts()['Yes']
            except:
                yes_count1 = 0
            try:
                no_count1 = data_temp[data.columns[len(data.columns)-1]].value_counts()['No']
            except:
                no_count1 = 0
            entropy_branch = entropy_cont('play',data=data_temp)          # Child entropy
            weight = no_list[j]/sum(no_list)      # Weight = no. of examples in that branch/total in parent
            child = child + entropy_branch*weight    # Calculating weighted entropy.
        info_gain.append(parent - child)
    # print(info_gain)
    return data.columns[info_gain.index(max(info_gain))+1]       # Because we are ignoring the 'day' column in our calculations

In [50]:
data.groupby('outlook').count()[data.columns[len(data.columns)-1]].index.tolist()

['Overcast', 'Rain', 'Sunny']

#### Testing on the dataset

In [51]:
best_split(data)

'outlook'

###### Split will take place on the 'outlook' column first

## TASK 2 : 
You have to work on the iris dataset and manually find out the splitting criteria for Petal Length column

In [52]:
iris = pd.read_csv('/content/drive/My Drive/Iris.csv')
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [53]:
# Replacing columns for ease
iris.drop(columns=['Id'],inplace=True)
new_column_name = {'SepalLengthCm':'SL','SepalWidthCm':'SW','PetalLengthCm':'PL','PetalWidthCm':'PW'}
iris = iris.rename(columns = new_column_name )
iris

,SL,SW,PL,PW,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


#### The Iris dataset (decision trees on numerical data)

#### Defining function for best split condition on the Petal Length column

In [0]:
# Column to work on : PL
# Sort dataset on PL
iris_sorted = iris.sort_values('PL')

def split_condition(data=iris_sorted):
  # Parent entropy
  parent_ent = entropy_cont('Species')

  # Iterating to find best condition of split on PL
  info_gain = []
  for val in iris_sorted['PL'][:-1]:
    # Splitting dataset
    data_less = iris_sorted[iris_sorted['PL']<=val]
    data_more = iris_sorted[iris_sorted['PL']>val]

    # Calculate entropy of each child dataset
    e1 = entropy_cont('Species',data=data_less)
    e2 = entropy_cont('Species',data=data_more)

    # Calculate weighted entropy 
    weighted_entropy = (len(data_less)*e1 + len(data_more)*e2)/len(iris_sorted)
    info_gain.append(parent_ent - weighted_entropy)
  return iris_sorted['PL'][info_gain.index(max(info_gain))]

#### The split condition on the Petal Length column

In [55]:
print("Splitting condition on Petal Length : Petal Length(PL) > ",split_condition())

Splitting condition on Petal Length : Petal Length(PL) >  1.5
